In [ ]:
import itertools
import math
import re
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.rcParams["figure.figsize"] = (12, 8)
# plt.rcParams["figure.autolayout"] = True # equivalent to .tight_layout(); breaks some legends
plt.rcParams["figure.constrained_layout.use"] = True  # Experimental

sns.set_style("white")
sns.set_context("notebook")

In [ ]:
# exporter colors and prettier ESC names from config
exp_colors = snakemake.config["plotting"]["exporter_colors"]
esc_pretty_names = snakemake.config["plotting"]["esc_pretty_names"]
esc_markers = snakemake.config["plotting"]["esc_markers"]

# Select data of interest
df = pd.read_csv(snakemake.input[0], sep=";")

df = df.loc[
    (df["scenario"] == snakemake.wildcards["scenario"])
    & (df["year"] == int(snakemake.wildcards["year"]))
    & (df["subcategory"].isin(["Curtailed electricity", 
                "Total produced electricity",
                "Energy surplus factor",
                "Cost per MWh delivered"]))
].drop(columns=["scenario","year","wacc","importer"])

# Rearrange for easier handling
df = df.pivot(index=["esc","exporter"],
        columns="subcategory",
        values="value")

# Calculate curtailement to be plotted
df["Curtailed electricity [%]"] = df["Curtailed electricity"]/df["Total produced electricity"]*100

# Reset for nicer row iteration
df = df.reset_index()

f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=plt.rcParams["figure.figsize"])

for idx, row in df.iterrows():
    # left plot
    ax1.scatter(
        x = row["Cost per MWh delivered"],
        y = row["Energy surplus factor"],
        marker=esc_markers[row["esc"]],
        color=exp_colors[row["exporter"]],
        s=50
    )
    
    # right plot
    ax2.scatter(
        x = row["Curtailed electricity [%]"],
        y = row["Energy surplus factor"],
        marker=esc_markers[row["esc"]],
        color=exp_colors[row["exporter"]],
        s=50
    )

ax1.set_ylabel("Energy surplus factor")
ax1.set_xlabel("EUR per MWh delivered")
ax2.set_xlabel("Curtailed electricity [%]")

# Construct a single legend for both plots manually
legend_esc = [
    matplotlib.lines.Line2D(
        [], [], color="None", linestyle="None", markersize=0, label="ESC", marker=None
    )
]
legend_esc += [
    matplotlib.lines.Line2D(
        [],
        [],
        color="white",
        linestyle="None",
        markersize=10,
        markeredgecolor="black",
        label=esc_pretty_names[n],
        marker=m,
    )
    for n, m in esc_markers.items()
]

legend_exp = [
    matplotlib.lines.Line2D(
        [],
        [],
        color="None",
        linestyle="None",
        markersize=0,
        label="Exporter",
        marker=None,
    )
]
legend_exp += [
    matplotlib.lines.Line2D(
        [], [], linestyle="None", markersize=10, marker=".", label=n, color=c
    )
    for n, c in exp_colors.items()
]
# Hack: Add dummy entries to have two columns (one ESC, one Exporter) in legend
legend_exp += [
    matplotlib.lines.Line2D(
        [], [], linestyle="None", markersize=10, marker=".", label="", color="white", alpha=0,
    )
    for x in range(len(esc_markers)-len(exp_colors))
]

ax3.legend(handles=legend_esc+legend_exp, ncol=2,
           loc="center left",
           framealpha=0.4,
          )

ax1.grid(which="major", axis="both", visible=True)
ax2.grid(which="major", axis="both", visible=True)
ax3.axis("off") # Don't show legend axis

# Save figure for all file types (PDF, PNG, ...)
for fp in snakemake.output["figures"]:
    plt.savefig(fp, dpi=snakemake.config["plotting"]["dpi"])